# Add gaussian noise to a `Target`

You may want to generate a target with noise on the parameters without having to run the whole dataset + fitting process. 

The `apply_gaussian_noise` method of the ``Target`` object is made for that.

### Noise model follows the modeldag approach.

The errors are drawn using the [modeldag](https://modeldag.readthedocs.io/) structure: `{"name": {"func": , "kwargs":{}}, }`

This model specifies how the **parameter error** is drawn. Then, the "observed" parameter will be drawn from a normal distribution centered on the "true" parameter with the parameter error as width, so: ``param_obs = np.normal.rvs(loc=param_true, scale=param_err)``


***
## Most simple case, random noise on 1 parameter


### Step 0: create a timeserie target based on the "v19-2005bf" sncosmo template.

In [1]:
import skysurvey
import numpy as np

In [3]:
target = skysurvey.TSTransient(template="v19-2005bf")
# specify the absolute magnitude of the target (nan by default)
target.set_magabs([-18, 1])

# draw 1000 target, use inplace=True to change self.data
_ = target.draw(1_000, inplace=True)
target.data.head(3)

,z,t0,magabs,ra,dec,magobs,amplitude,template
0,0.04635,56080.234375,-18.542274,217.788696,80.624512,18.094978,3.164150e-15,v19-2005bf
1,0.04025,56033.644531,-18.581287,317.605774,7.081192,17.740023,4.387723e-15,v19-2005bf
2,0.04865,56147.906250,-14.416636,145.240326,-56.762920,22.329351,6.404838e-17,v19-2005bf


### Step 1: create an errormodel for `magobs`

Let's draw error on `magobs` assuming they are $0.1 ± 0.01$:

In [4]:
errormodel = {"magobs": {"func": np.random.normal, "kwargs":{"loc":0.1, "scale":0.01}} }

### Step 2: apply gaussian noise to the target's parameter

You will have a warning message, because the code doesn't know who to draw the error. 

In [5]:
noisy_target = target.apply_gaussian_noise(errormodel)
noisy_target.data.head(1)

,z,t0,magabs,ra,dec,magobs_true,amplitude,template,magobs_err,magobs
0,0.04635,56080.234375,-18.542274,217.788696,80.624512,18.094978,3.164150e-15,v19-2005bf,0.110416,18.045027


### What happened ?

For each parameter given in the errormodel (here, only `magobs`):

  1. rename `{parameter}` as `{parameter}_true`
  2. draw `{parameter}_{err}` following the errormodel
  4. draw `{parameter}` from a normal distribution centered on `{parameter}_true` with a width of `{parameter}_{err}`